extract coordinates from .h5 file to .csv file\
you can use this file to render at blender.\
I used blender 5.0.1

In [19]:
import os
import re
import h5py
import numpy as np

In [20]:
def make_chains_eq(N, number_of_chains, is_ring=False):
    chains = []
    
    base = N // number_of_chains
    remainder = N % number_of_chains
    
    start = 0
    
    for i in range(number_of_chains):
        size = base
        if i == number_of_chains - 1:
            size += remainder
        
        end = start + size
        
        if i == number_of_chains - 1:
            chains.append((start, None, is_ring))
        else:
            chains.append((start, end, is_ring))
        
        start = end
    
    return chains

In [21]:
def load_full_trajectory(
    folder_path,
    number_of_chains,
    output_path,
    is_ring=False,
    frame_stride=1,
):
    """
    folder_path: trajectory_xxx folder
    number_of_chains: same value used in simulation --> must be same!!!
    is_ring: same flag used in simulation --> True for ring and False for linear DNA
    frame_stride: e.g. 10 → take every 10th frame
    """

    #sort block files
    block_files = [f for f in os.listdir(folder_path)
                   if f.startswith("blocks_") and f.endswith(".h5")]

    block_files.sort(key=lambda f: int(re.search(r"blocks_(\d+)-", f).group(1)))

    #total frames calculation
    total_frames = 0
    frame_counts = []

    for file in block_files:
        with h5py.File(os.path.join(folder_path, file), "r") as f:
            n = len(f.keys())
            frame_counts.append(n)
            total_frames += n

    #total frames number after striding!
    total_frames = total_frames // frame_stride

    #check for N
    with h5py.File(os.path.join(folder_path, block_files[0]), "r") as f:
        first_key = sorted(f.keys(), key=lambda x: int(x))[0]
        N = f[first_key]["pos"].shape[0]

    #same_chains_with_simulation
    chains = make_chains_eq(N, number_of_chains, is_ring)
    # chains = [(start, end, is_ring), ...]

    #length of each chain
    chain_lengths = []
    for start, end, _ in chains:
        if end is None:
            end = N
        chain_lengths.append(end - start)

    max_len = max(chain_lengths)


    data = np.zeros(
        (total_frames, number_of_chains, max_len, 3),
        dtype=np.float32
    )


    frame_index = 0
    global_frame_counter = 0

    for file in block_files:
        with h5py.File(os.path.join(folder_path, file), "r") as f:
            frame_keys = sorted(f.keys(), key=lambda x: int(x))

            for key in frame_keys:

                #striding!
                if global_frame_counter % frame_stride != 0:
                    global_frame_counter += 1
                    continue

                pos = f[key]["pos"][:]

                for c, (start, end, _) in enumerate(chains):
                    if end is None:
                        end = N
                    chain = pos[start:end]
                    data[frame_index, c, :chain.shape[0]] = chain

                frame_index += 1
                global_frame_counter += 1

    np.save(output_path, data)

    print("Saved:", output_path)
    print("Shape:", data.shape)

In [23]:
load_full_trajectory(
    folder_path="trajectory_20260223_183802",
    number_of_chains=4,
    output_path="spherical_multiple_chrom_dynamics_20260223.npy",
    is_ring = False,
    frame_stride = 10
)

#number_of_chains has to match with the one you used at polychrom_simu_get_h5.ipynb
#frame_stride : steps for frame analysis (getting coordinates)

Saved: spherical_multiple_chrom_dynamics_20260223.npy
Shape: (1000, 4, 2500, 3)


In [ ]:
#shape = (total_frames, number_of_chains, max_len, xyz)